
### 1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>. You have to use data.csv file for this assignment
### 2. Code the model to classify data like below image. You can use any number of units in your Dense layers.

<img src='https://i.imgur.com/33ptOFy.png'>



# <font color='red'> <b>3. Writing Callbacks </b> </font>
## You have to implement the following callbacks
-  Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.Do not use tf.keras.metrics for calculating AUC and F1 score.

- Save your model at every epoch if your validation accuracy is improved from previous epoch. 

- You have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
- If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

- You have to stop the training if your validation accuracy is not increased in last 2 epochs.

- Use tensorboard for every model and analyse your scalar plots and histograms. (you need to upload the screenshots and write the observations for each model for evaluation)



<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score, f1_score
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model, Sequential
import random as rn
from tensorflow import keras
import datetime, os

In [2]:
data = pd.read_csv(r'data.csv')

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['f1','f2']],data['label'], test_size=0.2)

In [4]:
from sklearn.metrics import recall_score
class LossHistory(tf.keras.callbacks.Callback):
    def __init__(self,validation_data):
        self.x_test = validation_data[0]
        self.y_test= validation_data[1]
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': [],'val_recall': [], 'val_auc':[]}
    def on_epoch_end(self, epoch, logs={}):
        true_positives=0
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['accuracy'].append(logs.get('accuracy'))
      
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history['val_accuracy'].append(logs.get('val_accuracy'))
        # we can get a list of all predicted values at the end of the epoch
        # we can use these predicted value and the true values to calculate any custom evaluation score if it is needed for our model
        # Here we are taking log of all true positives and then taking average of it
        y_pred= self.model.predict(self.x_test)
        y_label_pred=np.argmax(y_pred,axis=1)
        custom_score = np.log(np.sum(y_test== y_label_pred))/len(y_test)
        
        #we can also calcualte predefined metrics such as precison, recall, etc. using callbacks 
        recall = recall_score(y_test,y_label_pred,average='micro')
        self.history['val_recall'].append(recall)
        print('custom_Score: ',np.round(custom_score,5),'Recall: ',recall)
        
        auc = roc_auc_score(y_test, y_pred, average='micro')
        self.history['val_recall'].append(recall)
        print('custom_Score: ',np.round(custom_score,5),'auc: ',auc)
        
def learning_rate_third_epoch(epoch,lr):
    if epoch % 3 == 0:
        lr = lr - (lr * 0.05)
        return lr
    return lr

class TerminateNaN(tf.keras.callbacks.Callback):
        
    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True
                
    



In [5]:

# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs


In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler

history_own=LossHistory(validation_data=[X_test,y_test])  
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=1, min_lr=0.0001)
LrScheduler = LearningRateScheduler(learning_rate_third_epoch,verbose=1)
filepath="model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.35, patience=2, verbose=1)
terminate_nan = TerminateNaN()
log_dir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [7]:
model_1 = Sequential([
            Dense(10,activation="tanh",input_shape=(2,),kernel_initializer=keras.initializers.RandomUniform(minval=-0, maxval=1)),
            Dense(20,activation="tanh",kernel_initializer=keras.initializers.RandomUniform(minval=-0, maxval=1)),
            Dense(25,activation="tanh",kernel_initializer=keras.initializers.RandomUniform(minval=-0, maxval=1)),
            Dense(15,activation="tanh",kernel_initializer=keras.initializers.RandomUniform(minval=-0, maxval=1)),
            Dense(10,activation="tanh",kernel_initializer=keras.initializers.RandomUniform(minval=-0, maxval=1)),
            Dense(1, activation='sigmoid')
            ])

In [8]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 20)                220       
                                                                 
 dense_2 (Dense)             (None, 25)                525       
                                                                 
 dense_3 (Dense)             (None, 15)                390       
                                                                 
 dense_4 (Dense)             (None, 10)                160       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,336
Trainable params: 1,336
Non-trainabl

In [9]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD')
model_1.compile(optimizer=optimizer,loss='BinaryCrossentropy',metrics=['accuracy'])
callbacks_list = [history_own,reduce_lr,LrScheduler,checkpoint,earlystop,terminate_nan,tensorboard_callback]
model_1.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=16,callbacks=callbacks_list)


Epoch 1: LearningRateScheduler setting learning rate to 0.009499999787658453.
Epoch 1/20
 997/1000 [============================>.] - ETA: 0s - loss: 0.6944 - accuracy: 0.5023custom_Score:  0.0019 Recall:  0.495
custom_Score:  0.0019 auc:  0.48712371237123714

Epoch 1: val_loss improved from inf to 0.69346, saving model to model_save\weights-01-0.4897.hdf5
1000/1000 [==============================] - 5s 3ms/step - loss: 0.6944 - accuracy: 0.5024 - val_loss: 0.6935 - val_accuracy: 0.4897 - lr: 0.0095

Epoch 2: LearningRateScheduler setting learning rate to 0.009499999694526196.
Epoch 2/20
 998/1000 [============================>.] - ETA: 0s - loss: 0.6935 - accuracy: 0.4984custom_Score:  0.0019 Recall:  0.495
custom_Score:  0.0019 auc:  0.5066924192419242

Epoch 2: val_loss improved from 0.69346 to 0.69311, saving model to model_save\weights-02-0.5077.hdf5
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6935 - accuracy: 0.4983 - val_loss: 0.6931 - val_accuracy: 0.507

In [10]:
%tensorboard --logdir logs/fits

Reusing TensorBoard on port 6006 (pid 1424), started 1:33:43 ago. (Use '!kill 1424' to kill it.)

In [21]:
del ./logs/ 

SyntaxError: invalid syntax (<ipython-input-21-f81d50a69e88>, line 1)

<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>




<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

# Note 
Make sure that you are plotting tensorboard plots either in your notebook or you can try to create a pdf file with all the tensorboard screenshots.Please write your analysis of tensorboard results for each model.
